In [1]:
!pip install crewai -r ../requirements.txt

In [2]:
import dotenv
assert dotenv.load_dotenv()

In [40]:
# Import required libraries
import os
from langchain_aws import ChatBedrock

# Set up the model ID for Claude
MODEL_ID3 = "meta.llama3-8b-instruct-v1:0"
MODEL_ID5 = "meta.llama3-70b-instruct-v1:0"
#MODEL_ID = "mistral.mistral-7b-instruct-v0:2"
MODEL_ID4 = "mistral.mixtral-8x7b-instruct-v0:1"
MODEL_ID2 = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Initialize the ChatBedrock instance
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0})
llm2 = ChatBedrock(model_id=MODEL_ID2, model_kwargs={'temperature': 0})
llm3 = ChatBedrock(model_id=MODEL_ID3, model_kwargs={'temperature': 0})
llm4 = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.7})

In [41]:
# from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it on a scale from 2 to 5 as either being a closed question (2), open question (3), very open question (4) or bananas (5).

Do not respond with more than one number.
A question should be classified as 'bananas', if it is not related to PIRLS 2021.

<question>
{question}
</question>

Classification:"""
    )
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "Give me a chocolate cake recipe."})

'5'

In [90]:
closed_chain = PromptTemplate.from_template(
    """You are an expert in PIRLS 2021. \
Your answer should not be longer than 1 or 2 paragraphs". \
The answer to the user's question in markdown format. Each paragraph starts with an emoji and a heading in capital letters. Use tables to increase readability. \
Respond to the following question:

Question: {question}
Answer:"""
) | llm
open_chain = PromptTemplate.from_template(
    """You are an expert in PIRLS 2021. \
Your answer should not be longer than 3 or 4 paragraphs". \
The answer to the user's question in markdown format. Each paragraph starts with an emoji and a heading in capital letters. Use tables to increase readability. \
Respond to the following question:

Question: {question}
Answer:"""
) | llm
very_open_chain = PromptTemplate.from_template(
    """You are an expert in PIRLS 2021. \
Your answer should not be longer than 5 paragraphs". \
The answer to the user's question in markdown format. Each paragraph starts with an emoji and a heading in capital letters. Use tables to increase readability. \
Respond to the following question:

Question: {question}
Answer:"""
) | llm
out_of_scope_chain = PromptTemplate.from_template(
    """You are a communications specialist. \
ALWAYS start the answer with a respectful information that the question is out of scope.
ALWAYS write your answer in the form of a 4 line poem that playfully picks up the user question but talks about reading. \
Make excessive use of emojis.
ALWAYS end with a request to ask a question related to PIRLS 2021, give a one sentence explanation what PIRLS 2021.
ALWAY end the answer with a link collection and list https://pirls2021.org/ as an additional source for general information.

Question: {question}
Answer:"""
) | llm

In [91]:
def route(info):
    if "2" in info["topic"].lower():
        return closed_chain
    elif "3" in info["topic"].lower():
        return open_chain
    elif "4" in info["topic"].lower():
        return very_open_chain
    else:
        return out_of_scope_chain

In [92]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    route
)

In [93]:
full_chain.invoke({"question": "Give me a chocolate cake recipe."})

AIMessage(content="I apologize, but that's not within my scope.\n\n📚 Recipes are sweet, but reading's a treat 🍰\nBooks nourish minds, better than what we eat 🧠\nPages of stories, a feast for the soul 📖\nReading's the cake that makes us feel whole 🎂\n\nInstead of cake recipes, may I suggest asking about PIRLS 2021? It's an international study that assesses reading literacy among fourth-grade students worldwide.\n\nFor more information:\nhttps://pirls2021.org/", additional_kwargs={'usage': {'prompt_tokens': 144, 'completion_tokens': 125, 'total_tokens': 269}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 144, 'completion_tokens': 125, 'total_tokens': 269}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-37b4b618-26e7-447b-a208-98f2e9cbf692-0', usage_metadata={'input_tokens': 144, 'output_tokens': 125, 'total_tokens': 269})